In [1]:
import json 
import numpy as np 
import os

In [2]:
def export_conv_weights(name, wts_coef, bias_coef):
    print("Exporting weights for {}\n\t{}\n\t{}".format(name, 
          os.path.join('exports', "{}_conv.data".format(name)), 
          os.path.join('exports', "{}_bias.data".format(name))))
    
    print("\n")
    
    # [kernel_width, kernel_height, input_feature_channels, output_feature_channels]
    print("\tOriginal weights shape {}".format(wts_coef.shape))
    if bias_coef is not None:
        # [output_feature_channels]
        print("\tOriginal bias shape {}".format(bias_coef.shape))
    
    # [output_feature_channels, kernel_width, kernel_height, input_feature_channels]
    wts_coef = wts_coef.transpose(3, 0, 1, 2)
    print("\tReshaped weights shape {}".format(wts_coef.shape))    
    wts_coef.tofile(os.path.join('exports', "{}_conv.data".format(name)))    
    
    if bias_coef is not None:
        bias_coef = np.squeeze(bias_coef)
        print("\tReshaped bias_coef shape {}".format(bias_coef.shape))    
        bias_coef.tofile(os.path.join('exports', "{}_bias.data".format(name)))    
        
    print("\n")

In [3]:
def export_dense_weights(name, wts_coef, bias_coef, kernel_width, kernel_height):
    """
    A fully connected layer in a Convolutional Neural Network (CNN) is one where every input channel is connected 
    to every output channel. The kernel width is equal to the width of the source image, and the 
    kernel height is equal to the height of the source image. The width and height of the output is 1 x 1.
    
    A fully connected layer takes an MPSImage object with dimensions 
    source.width x source.height x Ni, convolves it with Weights[No][source.width][source.height][Ni], 
    and produces a 1 x 1 x No output.
    
    Thus, the following conditions must be true:
    - kernelWidth == source.width
    - kernelHeight == source.height
    - clipRect.size.width == 1
    - clipRect.size.height == 1
    
    You can think of a fully connected layer as a matrix multiplication where the image is 
    flattened into a vector of length source.width*source.height*Ni, and the weights are arranged in a 
    matrix of dimension No x (source.width*source.height*Ni) to produce an output vector of length No.
    
    The value of the strideInPixelsX, strideInPixelsY, and groups properties must be 1. 
    The offset property is not applicable and it is ignored. Because the clip rectangle is 
    clamped to the destination image bounds, if the destination is 1 x 1, you do not need to set the 
    clipRect property.
    """
    print("Exporting weights for {}\n\t{}\n\t{}".format(name, 
          os.path.join('exports', "{}_conv.data".format(name)), 
          os.path.join('exports', "{}_bias.data".format(name))))
        
    input_feature_channels = int(wts_coef.shape[0] / kernel_width / kernel_height) 
    output_feature_channels = wts_coef.shape[-1]            
    
    # [kernel_width, kernel_height, input_feature_channels, output_feature_channels]
    print("\tOriginal weights shape {}".format(wts_coef.shape))
    
    #wts_coef = np.reshape(wts_coef, [kernel_width, kernel_height, input_feature_channels, output_feature_channels])    
    wts_coef = np.reshape(wts_coef, [kernel_width, kernel_height, -1, output_feature_channels])    
        
    if bias_coef is not None:
        # [output_feature_channels]
        print("\tOriginal bias shape {}".format(bias_coef.shape))
    
    # [output_feature_channels, kernel_width, kernel_height, input_feature_channels]
    wts_coef = wts_coef.transpose(3, 0, 1, 2)
    print("\tReshaped weights shape {}".format(wts_coef.shape))    
    wts_coef.tofile(os.path.join('exports', "{}_conv.data".format(name)))    
    
    if bias_coef is not None:
        bias_coef = np.squeeze(bias_coef)
        print("\tReshaped bias_coef shape {}".format(bias_coef.shape))    
        bias_coef.tofile(os.path.join('exports', "{}_bias.data".format(name)))   
        
    print("\n")

In [4]:
from keras.models import model_from_json
import keras 

Using TensorFlow backend.


In [5]:
GENERATOR_FILE = '/Users/joshua.newnham/Dropbox/my projects/DL4Swift/Hands-On-Deep-Learning-with-Swift/Chapter06/WIP/Notebooks/tf_weights/dcgan_generator.h5'
generator = keras.models.load_model(GENERATOR_FILE)

In [6]:
DISCRIMINATOR_FILE = '/Users/joshua.newnham/Dropbox/my projects/DL4Swift/Hands-On-Deep-Learning-with-Swift/Chapter06/WIP/Notebooks/tf_weights/dcgan_discriminator.h5'
discriminator = keras.models.load_model(DISCRIMINATOR_FILE)
discriminator.summary()



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
d_conv_1 (Conv2D)            (None, 14, 14, 64)        1664      
_________________________________________________________________
activation_4 (Activation)    (None, 14, 14, 64)        0         
_________________________________________________________________
d_conv_2 (Conv2D)            (None, 7, 7, 128)         204928    
_________________________________________________________________
activation_5 (Activation)    (None, 7, 7, 128)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 6272)              0         
_________________________________________________________________
d_dense_1 (Dense)            (None, 256)               1605888   
__________

/anaconda3/envs/dl4swift27/lib/python2.7/site-packages/keras/engine/saving.py:327: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


In [7]:
flatted_input_kernel_width = None
flatted_input_kernel_height = None

for layer in discriminator.layers:        
    print(layer.name)
    if "flatten" in layer.name:        
        flatted_input_kernel_width = layer.input_shape[1] 
        flatted_input_kernel_height = layer.input_shape[2] 
        
    if len(layer.get_weights()) > 0:        
        name = layer.name         
        wts = layer.get_weights()        
        
        if "conv" in name:
            export_conv_weights(layer.name, wts[0], wts[1] if len(wts) == 2 else None)        
        else:
            export_dense_weights(layer.name, wts[0], wts[1] if len(wts) == 2 else None, 
                                flatted_input_kernel_width, flatted_input_kernel_height)        
            # after the initial pass (from cnn to fcn); flattern the kernel down to 1x1 
            # i.e. update the flatted_input_kernel_DIM to have the kernel width and height of 1 
            flatted_input_kernel_width, flatted_input_kernel_height = 1, 1

input_1
d_conv_1
Exporting weights for d_conv_1
	exports/d_conv_1_conv.data
	exports/d_conv_1_bias.data


	Original weights shape (5, 5, 1, 64)
	Original bias shape (64,)
	Reshaped weights shape (64, 5, 5, 1)
	Reshaped bias_coef shape (64,)


activation_4
d_conv_2
Exporting weights for d_conv_2
	exports/d_conv_2_conv.data
	exports/d_conv_2_bias.data


	Original weights shape (5, 5, 64, 128)
	Original bias shape (128,)
	Reshaped weights shape (128, 5, 5, 64)
	Reshaped bias_coef shape (128,)


activation_5
flatten_1
d_dense_1
Exporting weights for d_dense_1
	exports/d_dense_1_conv.data
	exports/d_dense_1_bias.data
	Original weights shape (6272, 256)
	Original bias shape (256,)
	Reshaped weights shape (256, 7, 7, 128)
	Reshaped bias_coef shape (256,)


activation_6
d_dense_2
Exporting weights for d_dense_2
	exports/d_dense_2_conv.data
	exports/d_dense_2_bias.data
	Original weights shape (256, 1)
	Original bias shape (1,)
	Reshaped weights shape (1, 1, 1, 256)
	Reshaped bias_coef shape ()


In [8]:
for layer in generator.layers:
    print(layer.name)

input_2
g_dense_1
activation_8
reshape_1
up_sampling2d_1
g_conv_1
activation_9
up_sampling2d_2
g_conv_2
activation_10


In [9]:
flatted_input_kernel_width = 100
flatted_input_kernel_height = 1

for layer in generator.layers:  
    print("{} output shape {}".format(layer.name, layer.output.shape))
    
    if "flatten" in layer.name:
        flatted_input_kernel_width = layer.input_shape[1] 
        flatted_input_kernel_height = layer.input_shape[2] 
        
    if len(layer.get_weights()) > 0:        
        name = layer.name         
        wts = layer.get_weights()
        
        if "conv" in name:
            export_conv_weights(layer.name, wts[0], wts[1] if len(wts) == 2 else None)        
        else:
            export_dense_weights(layer.name, wts[0], wts[1] if len(wts) == 2 else None, 
                                flatted_input_kernel_width, flatted_input_kernel_height)        
            # after the initial pass (from cnn to fcn); flattern the kernel down to 1x1 
            # i.e. update the flatted_input_kernel_DIM to have the kernel width and height of 1 
            flatted_input_kernel_width, flatted_input_kernel_height = 1, 1

input_2 output shape (?, 100)
g_dense_1 output shape (?, 6272)
Exporting weights for g_dense_1
	exports/g_dense_1_conv.data
	exports/g_dense_1_bias.data
	Original weights shape (100, 6272)
	Original bias shape (6272,)
	Reshaped weights shape (6272, 100, 1, 1)
	Reshaped bias_coef shape (6272,)


activation_8 output shape (?, 6272)
reshape_1 output shape (?, 7, 7, 128)
up_sampling2d_1 output shape (?, 14, 14, 128)
g_conv_1 output shape (?, 14, 14, 64)
Exporting weights for g_conv_1
	exports/g_conv_1_conv.data
	exports/g_conv_1_bias.data


	Original weights shape (5, 5, 128, 64)
	Original bias shape (64,)
	Reshaped weights shape (64, 5, 5, 128)
	Reshaped bias_coef shape (64,)


activation_9 output shape (?, 14, 14, 64)
up_sampling2d_2 output shape (?, 28, 28, 64)
g_conv_2 output shape (?, 28, 28, 1)
Exporting weights for g_conv_2
	exports/g_conv_2_conv.data
	exports/g_conv_2_bias.data


	Original weights shape (5, 5, 64, 1)
	Original bias shape (1,)
	Reshaped weights shape (1, 5, 5, 64)
	